In [7]:
import torch
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm 
import numpy as np
import pandas as pd 
from glob import glob
import random
import json

In [2]:
df = pd.read_csv("/data/healthy-ml/scratch/qixuanj/generative_validation/ALIA/data/iwildcam_v2.0/train_extra_subset.csv")

In [3]:
df

,split,location_remapped,location,sequence_remapped,seq_id,y,category_id,datetime,filename,image_id
0,test,24,53,10116,30200b20-7d42-11eb-8fb5-0242ac1c0002,24,71,2015-02-07 10:59:15.000,90b4425e-21bc-11ea-a13a-137349068a90.jpg,90b4425e-21bc-11ea-a13a-137349068a90
1,test,24,53,10116,30200b20-7d42-11eb-8fb5-0242ac1c0002,24,71,2015-02-07 10:59:16.000,89e6a57a-21bc-11ea-a13a-137349068a90.jpg,89e6a57a-21bc-11ea-a13a-137349068a90
2,test,24,53,10116,30200b20-7d42-11eb-8fb5-0242ac1c0002,24,71,2015-02-07 10:59:17.000,871db9aa-21bc-11ea-a13a-137349068a90.jpg,871db9aa-21bc-11ea-a13a-137349068a90
3,test,24,53,10116,30200b20-7d42-11eb-8fb5-0242ac1c0002,24,71,2015-02-07 10:59:37.000,910744cc-21bc-11ea-a13a-137349068a90.jpg,910744cc-21bc-11ea-a13a-137349068a90
4,test,24,53,10116,30200b20-7d42-11eb-8fb5-0242ac1c0002,24,71,2015-02-07 10:59:38.000,951b2f92-21bc-11ea-a13a-137349068a90.jpg,951b2f92-21bc-11ea-a13a-137349068a90
...,...,...,...,...,...,...,...,...,...,...
2219,test,315,540,35393,9904d0ea-21bc-11ea-a13a-137349068a90,24,71,2015-02-03 14:09:02.000,96aa86aa-21bc-11ea-a13a-137349068a90.jpg,96aa86aa-21bc-11ea-a13a-137349068a90
2220,test,315,540,35393,9904d0ea-21bc-11ea-a13a-137349068a90,24,71,2015-02-03 14:09:03.000,98dc43fa-21bc-11ea-a13a-137349068a90.jpg,98dc43fa-21bc-11ea-a13a-137349068a90
2221,test,315,540,35393,9904d0ea-21bc-11ea-a13a-137349068a90,24,71,2015-02-03 14:09:33.000,97c17a94-21bc-11ea-a13a-137349068a90.jpg,97c17a94-21bc-11ea-a13a-137349068a90
2222,test,315,540,35393,9904d0ea-21bc-11ea-a13a-137349068a90,24,71,2015-02-03 14:09:34.000,87f2bb50-21bc-11ea-a13a-137349068a90.jpg,87f2bb50-21bc-11ea-a13a-137349068a90


In [11]:
with open("/mnt/scratch-lids/scratch/qixuanj/iwildcam/iwildcam2020_train_annotations.json", "r") as f: 
    train_data = json.load(f)

In [13]:
with open("/mnt/scratch-lids/scratch/qixuanj/iwildcam/iwildcam2020_test_information.json", "r") as f: 
    test_data = json.load(f)

In [15]:
train_data.keys()

dict_keys(['annotations', 'images', 'categories', 'info'])

In [17]:
train_data['annotations'][0]

{'count': 1,
 'image_id': '96b00332-21bc-11ea-a13a-137349068a90',
 'id': 'a292dd3c-21bc-11ea-a13a-137349068a90',
 'category_id': 73}

In [18]:
train_data['images'][0]

{'seq_num_frames': 1,
 'location': 267,
 'datetime': '2013-08-08 11:45:00.000',
 'id': '96b00332-21bc-11ea-a13a-137349068a90',
 'frame_num': 1,
 'seq_id': '96b004ea-21bc-11ea-a13a-137349068a90',
 'width': 1795,
 'height': 1222,
 'file_name': '96b00332-21bc-11ea-a13a-137349068a90.jpg'}

In [19]:
train_data['categories'][0]

{'count': 78981, 'id': 0, 'name': 'empty'}

In [22]:
test_data.keys()

dict_keys(['images', 'categories', 'info'])

In [23]:
test_data['images'][0]

{'seq_num_frames': 1,
 'location': 84,
 'datetime': '2013-08-02 21:44:00.000',
 'id': '88037cce-21bc-11ea-a13a-137349068a90',
 'frame_num': 1,
 'seq_id': '88037e90-21bc-11ea-a13a-137349068a90',
 'width': 2771,
 'height': 1779,
 'file_name': '88037cce-21bc-11ea-a13a-137349068a90.jpg'}

In [42]:
# seq_id is not unique, file_name is 
all_train_seq_ids = []
for i in range(len(train_data['images'])): 
    all_train_seq_ids.append(train_data['images'][i]['file_name'])

In [43]:
print(len(all_train_seq_ids))
all_train_seq_ids = np.unique(np.array(all_train_seq_ids))
print(len(all_train_seq_ids))

217959
217959


In [46]:
all_train_seq_ids = [x.split('.')[0] for x in all_train_seq_ids]

In [51]:
# Check if subset is in downloaded dataset -> yes to all 
files = ["train_subset.csv", "train_extra_subset.csv", "val_subset.csv", "test_subset.csv",]
for file in files:
    df = pd.read_csv(f"/data/healthy-ml/scratch/qixuanj/generative_validation/ALIA/data/iwildcam_v2.0/{file}")
    shared = np.intersect1d(df['image_id'].values, np.array(all_train_seq_ids))
    print(file)
    print(len(df['image_id'].unique()))
    print(len(shared))

train_subset.csv
6052
6052
train_extra_subset.csv
2224
2224
val_subset.csv
2826
2826
test_subset.csv
8483
8483


In [52]:
from wilds.common.metrics.all_metrics import Accuracy, Recall, F1

/data/healthy-ml/scratch/qixuanj/anaconda3/envs/control/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
from collections import Counter
import wilds

In [55]:
def get_counts(labels):
    values, counts = np.unique(labels, return_counts=True)
    sorted_tuples = zip(*sorted(zip(values, counts))) # this just ensures we are getting the counts in the sorted order of the keys
    values, counts = [ list(tuple) for tuple in  sorted_tuples]
    fracs   = 1 / torch.Tensor(counts)
    return fracs / torch.max(fracs)

class Wilds:
    """
    Wrapper for the WILDS dataset.
    """
    def __init__(self, root="/work/datasets", split='train', transform=None):
        self.dataset = wilds.get_dataset(dataset="iwildcam", root_dir=root)#, download=True)
        self.split_dataset = self.dataset.get_subset(split, transform=transform)
        self._metadata_fields = self.split_dataset.metadata_fields
        self.classes = list(range(self.split_dataset.n_classes))
        self.df = pd.DataFrame(self.split_dataset.metadata_array.numpy(), columns=self.split_dataset.metadata_fields)
        self.groups = self.df.location.values
        self.labels = self.targets = self.df.y.values
        self.class_weights = get_counts(self.labels)
        self.samples = [(i, l) for (i, l) in zip(self.df.sequence.values, self.labels)]
        # self.locations = [m[0] for m in self.split_dataset._metadata_array]
        # location = LOCATION_MAP_INV[1] if split == 'test' else LOCATION_MAP_INV[0]
        # self.location_idxs = np.where(np.array(self.locations) == location)[0]
        # self.groups = [location for _ in range(len(self.location_idxs))]

    def __len__(self):
        return len(self.split_dataset)

    def __getitem__(self, idx):
        # map the idx to the location idx (filter out all other locations)
        # idx = self.location_idxs[idx]
        img, label, metadata = self.split_dataset[idx]
        return img, label, self.groups[idx]

class WILDS:
    """
    Specific subset of WILDS containing 6 classes and 2 test locations.
    """
    def __init__(self, root='/mnt/scratch-lids/scratch/qixuanj/iwildcam/train', split='train', transform=None):
        self.root = root
        self.df = pd.read_csv(f'/data/healthy-ml/scratch/qixuanj/generative_validation/ALIA/data/iwildcam_v2.0/{split}_subset.csv')
        self.transform = transform
        self.class_names = sorted(self.df.y.unique())
        self.label_map = {j:i for i, j in enumerate(self.class_names)}
        self.labels = [self.label_map[i] for i in self.df.y]
        self.samples = [(os.path.join(root, i), l) for (i, l) in zip(self.df.filename, self.labels)]
        self.targets = [l for _, l in self.samples]
        self.classes = list(sorted(self.df.y.unique()))
        self.locations = self.df.location_remapped.unique()
        self.location_map = {j:i for i, j in enumerate(self.locations)}
        self.location_labels = [self.location_map[i] for i in self.df.location_remapped]
        self.groups = self.location_labels
        self.class_weights = get_counts(self.labels)
        self.group_names = self.locations
        self.class_names = ['background', 'cattle', 'elephants', 'impalas', 'zebras', 'giraffes', 'dik-diks']
        print(f"Num samples per class {Counter(self.labels)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)
        location = self.location_labels[idx]
        return img, label, location

    def inspect_location(self, location):
        assert location in self.locations
        location_df = self.df[self.df.location_remapped == location]
        fig, axs = plt.subplots(1, 2, figsize=(20, 10))
        idx = np.random.choice(list(range(len(location_df))))
        location_df['y'].value_counts().plot(kind='bar', ax=axs[0])
        axs[0].set_title(f'Location {location} (n={len(location_df)}) class counts')
        axs[1].imshow(Image.open(os.path.join(self.root, location_df.iloc[idx].filename)))
        axs[1].set_title(f'Location {location} (n={len(location_df)}) class {location_df.iloc[idx].y} (idx={idx})')
        axs[1].axis('off')
        plt.show()

    def inspect_class(self, class_idx):
        assert class_idx in self.classes
        class_df = self.df[self.df.y == class_idx]
        fig, axs = plt.subplots(1, 2, figsize=(20, 10))
        idx = np.random.choice(list(range(len(class_df))))
        class_df['location_remapped'].value_counts().plot(kind='bar', ax=axs[0])
        axs[0].set_title(f'Class {class_idx} (n={len(class_df)}) location counts')
        axs[1].imshow(Image.open(os.path.join(self.root, class_df.iloc[idx].filename)))
        axs[1].set_title(f'Class {class_idx} (n={len(class_df)}) location {class_df.iloc[idx].location_remapped} (idx={idx}) ({class_df.iloc[idx].filename})')
        axs[1].axis('off')
        plt.show()

    @staticmethod
    def eval(y_pred, y_true, prediction_fn=None):
        """
        Computes all evaluation metrics.
        Args:
            - y_pred (Tensor): Predictions from a model. By default, they are predicted labels (LongTensor).
                               But they can also be other model outputs such that prediction_fn(y_pred)
                               are predicted labels.
            - y_true (LongTensor): Ground-truth labels
            - metadata (Tensor): Metadata
            - prediction_fn (function): A function that turns y_pred into predicted labels
        Output:
            - results (dictionary): Dictionary of evaluation metrics
            - results_str (str): String summarizing the evaluation metrics
        """
        metrics = [
            Accuracy(prediction_fn=prediction_fn),
            Recall(prediction_fn=prediction_fn, average='macro'),
            F1(prediction_fn=prediction_fn, average='macro'),
        ]

        results = {}

        for i in range(len(metrics)):
            results.update({
                **metrics[i].compute(y_pred, y_true),
                        })

        results_str = (
            f"Average acc: {results[metrics[0].agg_metric_field]:.3f}\n"
            f"Recall macro: {results[metrics[1].agg_metric_field]:.3f}\n"
            f"F1 macro: {results[metrics[2].agg_metric_field]:.3f}\n"
        )

        return results, results_str

In [58]:
train_ds = WILDS(split="train")
train_extra_ds = WILDS(split="train_extra")
val_ds = WILDS(split="val")
test_ds = WILDS(split="test")

Num samples per class Counter({0: 2210, 3: 981, 1: 801, 4: 720, 6: 514, 5: 460, 2: 366})
Num samples per class Counter({3: 625, 1: 506, 6: 468, 0: 402, 2: 91, 4: 85, 5: 47})
Num samples per class Counter({0: 2006, 6: 244, 1: 206, 3: 140, 4: 119, 5: 58, 2: 53})
Num samples per class Counter({3: 4533, 2: 2003, 1: 1416, 6: 213, 4: 144, 0: 127, 5: 47})


6052